# Exercício 1

## Enunciado do Problema

Use a package **Criptography** para 

1. Criar um comunicação privada assíncrona entre um agente ***Emitter*** e um agente ***Receiver*** que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data). Usar uma cifra simétrica num modo **HMAC** que seja seguro contra ataques aos “nounces” .
    2. Os “nounces” são gerados por um gerador pseudo aleatório (PRG) construído por um função de hash em modo XOF.
    3. O par de chaves **cipher_key**, **mac_key** , para cifra e autenticação, é acordado entre agentes usando o protocolo ECDH com autenticação dos agentes usando assinaturas ECDSA.

## Descrição do Problema

Precisamos de garantir que uma comunicação entre um emitter(pessoa que envia) e um receiver(pessoa que recebe) ocorra de forma segura e privada. Para tal temos que definir os seguintes aspetos:

1. Autenticar o criptograma e os seus metadados através de uma cifra simétrica segura contra ataques aos "nounces".

- Gerar os "nounces" através de um PRG constituido por uma função de hash em modo XOF.

- Os dois agentes chegarem a um acordo quanto às chaves **cipher_key** e **mac_key** usando o protocolo **ECDH** com a autenticação usando assinaturas ECDSA.

## Abordagem

1. Para autenticar o criptograma, vamos utilizar a função HMAC.
2. Para garantir que a comunicação ocorra de forma privada precisamos de encriptar a mensagem e para isso vamos utilizar uma cifra simétrica.
3. Para garantir que a cifra simétrica é segura contra ataques aos nounces ou *replay attacks* podemos utilizar a função de hash para autenticar o nounce.
4. Para garantir aleatoriedade na geração dos "nounces", vamos utilizar uma função de hash em modo XOF (Extendable Output Function).
5. Para definirmos as chaves **cipher_key** e **mac_key** entre os dois agentes, vamos utilizar o protocolo **ECDH**(Elliptic-curve Diffie–Hellman) .
6. Para autenticar os agentes, vamos utilizar o algoritmo **ECDSA**(Elliptic Curve Digital Signature Algorithm).

## Código de resolução

...

In [1]:
import asyncio
# noinspection PyUnresolvedReferences
import os
from asyncio import StreamReader, StreamWriter

from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PublicFormat,
    load_pem_public_key,
)

In [2]:
NONCE_HASH_FUNCTION = hashes.SHAKE256
NONCE_HASH_SIZE = 16
DIGEST_SIZE = 128


def generate_random_nonce():
    """ Generate random nonce using XOF hash function """
    xof = hashes.Hash(NONCE_HASH_FUNCTION(DIGEST_SIZE))
    xof.update(os.urandom(NONCE_HASH_SIZE))

    return xof.finalize()

In [3]:
def encrypt_message(message: bytes, key: bytes, nonce: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CBC(nonce))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message) + encryptor.finalize()

    return ciphertext

In [4]:
def authenticate_message(message: bytes, key: bytes, nonce: bytes) -> bytes:
    hmac_algorithm = hmac.HMAC(key, hashes.SHA256())
    hmac_algorithm.update(nonce + message)
    tag = hmac_algorithm.finalize()

    return tag

In [5]:
def authenticate_and_encrypt_message(message: bytes, cipher_key: bytes, mac_key: bytes, nonce: bytes) -> (
        bytes, bytes, bytes):
    """Encrypt message with AES-256 in CBC mode and authenticate with HMAC-SHA256"""

    ciphertext = encrypt_message(message, cipher_key, nonce)

    tag = authenticate_message(ciphertext, mac_key, nonce)

    return ciphertext, tag, nonce


authenticate_and_encrypt_message(b'o' * 16, b'x' * 32, b'a' * 256, generate_random_nonce()[:16])

(b'\xf6\xb1\xb7\x0e\x12\x9e\xdc\x95\xa5\xfc\xd58\x19\xeex\xa3',
 b'|\xab\x8a7\xca\x86\xec\xbc(\x95V\t\x90\xd6\x19\n\xa3$\rbw\xdet)\x7f\x9c\xab\n\xf2\xd8Em',
 b'\xd5=\xbc\x13\xb0\x9d\xedF\x9cz\x1d\xf8]\x9c\xb8\xc8')

In [6]:
async def emitter(stream_reader: StreamReader, stream_writer: StreamWriter):
    # Generate ECDH (Elliptic-curve Diffie–Hellman) key pair
    ecdh_private_key = ec.generate_private_key(ec.SECP384R1())
    ecdh_public_key = ecdh_private_key.public_key()

    # Send ECDH public key to the receiver
    stream_writer.write(ecdh_public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo))

    # Receive receiver's ECDH public key
    receiver_public_key_bytes = await stream_reader.read()

    # Load receiver's public key
    receiver_public_key = load_pem_public_key(receiver_public_key_bytes)

    # Generate shared secret from ECDH key exchange
    shared_secret = ecdh_private_key.exchange(ec.ECDH(), receiver_public_key)

    # Derive cipher and MAC keys from shared secret using HKDF
    hkdf = HKDF(algorithm=hashes.SHA256(), length=64, salt=None, info=b'secret')
    hkdf_output = hkdf.derive(shared_secret)
    cipher_key, mac_key = hkdf_output[:32], hkdf_output[32:]

    nonce = generate_random_nonce()

    message = b'This is a secret message!'
    ciphertext, tag, nonce = authenticate_and_encrypt_message(message, cipher_key, mac_key, nonce)

    # Send encrypted message and authentication tag to the receiver
    sent_message = nonce + ciphertext + tag
    stream_writer.write(sent_message)

    print('Emitter sent message:', sent_message)
    print('Emitter encrypted message:', ciphertext)

    # Receive response from the receiver
    response = await stream_reader.read()

    print('Emitter received response:', response)

    stream_writer.close()


In [7]:
async def get_connection(host: str, port: int) -> (asyncio.StreamReader, asyncio.StreamWriter):
    return await asyncio.open_connection(host, port)


In [ ]:
LOCALHOST = '127.0.0.1'
PORT = 9001

(reader, writer) = await get_connection(LOCALHOST, PORT)

await emitter(reader, writer)

In [ ]:
def decrypt_message(ciphertext: bytes, key: bytes, nonce: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CBC(nonce))
    decryptor = cipher.decryptor()
    message = decryptor.update(ciphertext) + decryptor.finalize()

    return message

In [ ]:
def verify_message(message: bytes, key: bytes, nonce: bytes, tag: bytes) -> bool:
    hmac_algorithm = hmac.HMAC(key, hashes.SHA256())
    hmac_algorithm.update(nonce + message)
    hmac_algorithm.verify(tag)

    return True

In [ ]:

import os
from cryptography.hazmat.primitives.kdf.hkdf import HKDF


async def receiver(stream_reader: StreamReader, stream_writer: StreamWriter):
    # Generate ECDH (Elliptic-curve Diffie–Hellman) key pair
    ecdh_private_key = ec.generate_private_key(ec.SECP384R1())
    ecdh_public_key = ecdh_private_key.public_key()

    # Receive emitter's ECDH public key
    emitter_public_key_bytes = await stream_reader.read()

    # Send ECDH public key to the emitter
    stream_writer.write(ecdh_public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo))

    # Load emitter's public key
    emitter_public_key = load_pem_public_key(emitter_public_key_bytes)

    # Generate shared secret from ECDH key exchange
    shared_secret = ecdh_private_key.exchange(ec.ECDH(), emitter_public_key)

    # Derive cipher and MAC keys from shared secret using HKDF
    hkdf = HKDF(algorithm=hashes.SHA256(), length=64, salt=None, info=b'secret')
    hkdf_output = hkdf.derive(shared_secret)
    cipher_key, mac_key = hkdf_output[:32], hkdf_output[32:]

    # Receive nonce from the emitter
    nonce = await stream_reader.read(16)

    # Receive encrypted message and authentication tag from the emitter
    received_message = await stream_reader.read()

    print('Receiver received message:', received_message)

    ciphertext = received_message[16:-32]
    tag = received_message[-32:]

    print('Receiver received nonce:', nonce)
    print('Receiver received ciphertext:', ciphertext)
    print('Receiver received tag:', tag)

    # Decrypt message with AES-256 in CBC mode
    plaintext = decrypt_message(ciphertext, cipher_key, nonce)

    print('Receiver decrypted message:', plaintext)

    # Authenticate message with HMAC-SHA256
    verify_message(plaintext, tag, mac_key, nonce)



## Exemplos e testes da aplicação

...